In [10]:
from pathlib import Path
import pandas as pd

# ========= locate THA24.csv (robust) =========
NOTEBOOK_DIR = Path.cwd()
PROJECT_DIR, RAW_FILE = None, None

for parent in [NOTEBOOK_DIR, *NOTEBOOK_DIR.parents]:
    candidate = parent / "nta_public_transport_analysis" / "data" / "raw" / "THA24.csv"
    if candidate.exists():
        PROJECT_DIR = parent / "nta_public_transport_analysis"
        RAW_FILE = candidate
        break
if RAW_FILE is None:
    candidate = NOTEBOOK_DIR / "data" / "raw" / "THA24.csv"
    if candidate.exists():
        PROJECT_DIR = NOTEBOOK_DIR
        RAW_FILE = candidate
if RAW_FILE is None:
    raise FileNotFoundError("Could not find THA24.csv under nta_public_transport_analysis/data/raw/")

PROCESSED_DIR = PROJECT_DIR / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
print(f"Using RAW_FILE: {RAW_FILE}")

Using RAW_FILE: c:\Users\Sinah\OneDrive\Email attachments\Desktop\projects\nta_public_transport_analysis\data\raw\THA24.csv


In [11]:
# ========= brand colors (keys == EXACT raw labels) =========
mode_colors = {
    "Dublin Metro Bus":  "#ffe600",
    "Bus, excluding Dublin Metro": "#c2002f",
    "Rail": "#2e8539",
    "All public transport, excluding LUAS": "#5c068c",  # not used in mode_share
}

In [ ]:
# ========= load with exact headers =========
df = pd.read_csv(RAW_FILE)

# Build tidy table WITHOUT renaming your columns
out = pd.DataFrame({
    "Year": df["Year"].astype(int),
    "Mode of Transport": df["Mode of Transport"].astype(str),
    # Prefer numeric week column if available, else parse "Weeks of the year"
    "Week": (
        df["C01198V01436"].astype(int)
        if "C01198V01436" in df.columns and pd.api.types.is_numeric_dtype(df["C01198V01436"])
        else df["Weeks of the year"].astype(str).str.extract(r"(\d+)").iloc[:, 0].astype(int)
    ),
    "Journeys": pd.to_numeric(df["VALUE"], errors="coerce").fillna(0.0),
})

# Monday of ISO week for time axis
out["date"] = pd.to_datetime(
    out["Year"].astype(str) + "-W" + out["Week"].astype(str) + "-1",
    format="%G-W%V-%u",
    errors="coerce"
)

In [13]:
# ========= exports =========
weekly_trends = out.loc[:, ["date","Year","Week","Mode of Transport","Journeys"]].sort_values("date")
weekly_trends.to_csv(PROCESSED_DIR / "weekly_trends.csv", index=False)

yearly_totals = (
    out.groupby(["Year","Mode of Transport"], as_index=False)["Journeys"]
      .sum().sort_values(["Year","Mode of Transport"])
)
yearly_totals.to_csv(PROCESSED_DIR / "yearly_totals.csv", index=False)

# Mode share (exclude the aggregate row)
ms = yearly_totals[yearly_totals["Mode of Transport"] != "All public transport, excluding LUAS"].copy()
totals = ms.groupby("Year")["Journeys"].sum().rename("Year_Total")
mode_share = ms.merge(totals, on="Year", how="left")
mode_share["Mode_Share"] = (mode_share["Journeys"] / mode_share["Year_Total"]).clip(0, 1)
mode_share = mode_share.loc[:, ["Year","Mode of Transport","Mode_Share"]].sort_values(["Year","Mode of Transport"])
mode_share.to_csv(PROCESSED_DIR / "mode_share.csv", index=False)

# Color key for Tableau (“Edit Colors…”) — uses raw labels
present_modes = sorted(yearly_totals["Mode of Transport"].unique().tolist())
pd.DataFrame({
    "Mode of Transport": present_modes,
    "Hex": [mode_colors.get(m, "") for m in present_modes]
}).to_csv(PROCESSED_DIR / "mode_color_key.csv", index=False)

print(f"Saved to {PROCESSED_DIR}: weekly_trends.csv, yearly_totals.csv, mode_share.csv, mode_color_key.csv")

Saved to c:\Users\Sinah\OneDrive\Email attachments\Desktop\projects\nta_public_transport_analysis\data\processed: weekly_trends.csv, yearly_totals.csv, mode_share.csv, mode_color_key.csv
